In [14]:
import pandas as pd
import numpy as np

In [294]:
input_file = './data/data.xls'
data_raw = pd.read_excel(input_file)

In [295]:
data_raw.head().T

,0,1,2,3,4
肝气郁结证型系数,0.056,0.488,0.107,0.322,0.242
热毒蕴结证型系数,0.46,0.099,0.008,0.208,0.28
冲任失调证型系数,0.281,0.283,0.204,0.305,0.131
气血两虚证型系数,0.352,0.333,0.15,0.13,0.21
脾胃虚弱证型系数,0.119,0.116,0.032,0.184,0.191
肝肾阴虚证型系数,0.35,0.293,0.159,0.317,0.351
病程阶段,S4,S4,S4,S4,S4
TNM分期,H4,H4,H4,H4,H4
转移部位,R1,R1,R2,R2,R2R5
确诊后几年发现转移,J1,J1,J2,J1,J1


In [296]:
col_lst = list(data_raw.columns)

In [139]:
symbol_lst = ['A', 'B', 'C', 'D', 'E', 'F']
type_dict = {}

for i in range(6):
    type_dict[col_lst[i]] = symbol_lst[i]

In [10]:
from sklearn.cluster import KMeans

In [11]:
k = 4

In [211]:
def gen_cluster_df(Series, Sym):
    kmodel = KMeans(n_clusters=k, n_jobs=-1)
    kmodel.fit(np.array(Series).reshape((-1,1)))
    
    df_clus = pd.DataFrame(kmodel.cluster_centers_, columns=[Sym])
    s_count = pd.Series(kmodel.labels_, name=Sym+'n').value_counts()
    df_clus = pd.concat([df_clus, s_count], axis=1)
    
    df_clus = df_clus.sort_values(by=[Sym])
    df_clus = df_clus.reset_index(drop=True)
    df_clus[Sym] = df_clus[Sym].rolling(2).mean()
    df_clus.loc[[0],[Sym]] = 0

    
    return df_clus

In [212]:
df = pd.DataFrame()
for name in list(type_dict.keys()):
    df_clus = gen_cluster_df(data[name], type_dict[name])
    df = df.append(df_clus.T)

In [213]:
df

,0,1,2,3
A,0.0,0.178698,0.257724,0.351843
An,240.0,356.000000,281.000000,53.000000
B,0.0,0.154031,0.306200,0.564697
Bn,342.0,384.000000,194.000000,10.000000
C,0.0,0.202149,0.289061,0.423537
Cn,297.0,394.000000,204.000000,35.000000
D,0.0,0.173544,0.252731,0.359726
Dn,292.0,370.000000,224.000000,44.000000
E,0.0,0.153398,0.258200,0.376062
En,278.0,314.000000,245.000000,93.000000


In [216]:
from apriori import *
import time

In [279]:
input_file2 = './data/apriori.txt'
data = pd.read_csv(input_file2, header=None, dtype=object)

In [280]:
data.head()

,0,1,2,3,4,5,6
0,A2,B1,C3,D3,E1,F1,H1
1,A2,B1,C3,D3,E1,F1,H1
2,A2,B1,C3,D3,E1,F1,H1
3,A2,B1,C3,D3,E1,F1,H1
4,A2,B2,C3,D3,E1,F1,H1


In [281]:
ct = lambda x: pd.Series(1, index=x[pd.notnull(x)])

In [282]:
b = map(ct, data.values)

In [283]:
data = pd.DataFrame(list(b)).fillna(0)

In [292]:
support = 0.06
confidence = 0.75
ms = '---'

In [293]:
find_rule(data, support, confidence, ms)


正在进行第1次搜索...
数目：276...

正在进行第2次搜索...
数目：947...

正在进行第3次搜索...
数目：41...

结果为：
                    support  confidence
A3---F4---H4       0.078495    0.879518
C3---F4---H4       0.075269    0.875000
B2---F4---H4       0.062366    0.794521
C2---E3---D2       0.092473    0.754386
D2---F3---H4---A2  0.062366    0.753247


,support,confidence
A3---F4---H4,0.078495,0.879518
C3---F4---H4,0.075269,0.875000
B2---F4---H4,0.062366,0.794521
C2---E3---D2,0.092473,0.754386
D2---F3---H4---A2,0.062366,0.753247


In [339]:
s_R = data_raw[col_lst[8]]

In [340]:
s_R.unique()

array(['R1', 'R2', 'R2R5', 'R3', 'R1R3', 'R1R2R3R5', 'R5', 'R1R2R3',
       'R1R2R3R4R5', 'R2R3R5', 'R0', 'R2R3', 'R4', 'R1R2', 'R1R5', 'R1R4',
       'R2R4'], dtype=object)

In [341]:
import re


In [342]:
s_R = s_R.apply(lambda x: re.findall('(R\d)',x))

In [345]:
trans = lambda x: pd.Series(1, index=[i for i in x])

In [354]:
map_R = map(trans, s_R)

In [355]:
df_R = pd.DataFrame(list(map_R)).fillna(0)

In [371]:
df_new = data_raw[['病程阶段', '确诊后几年发现转移']]

In [372]:
map_new = map(ct, df_new.values)

In [373]:
df_new = pd.DataFrame(list(map_new)).fillna(0)

In [374]:
df_all = pd.concat([data, df_new, df_R], axis=1)

In [376]:
find_rule(df_all, support, confidence, ms)


正在进行第1次搜索...
数目：666...

正在进行第2次搜索...
数目：2537...

正在进行第3次搜索...
数目：292...

正在进行第4次搜索...
数目：10...

正在进行第5次搜索...
数目：0...

结果为：
                    support  confidence
J0---R0            0.548387    1.000000
H4---J0---R0       0.266667    1.000000
C2---J0---R0       0.240860    1.000000
D2---R0---J0       0.233333    1.000000
D2---J0---R0       0.233333    1.000000
A2---J0---R0       0.224731    1.000000
B2---R0---J0       0.217204    1.000000
B2---J0---R0       0.217204    1.000000
B1---J0---R0       0.193548    1.000000
R0---S4---J0       0.188172    1.000000
J0---S4---R0       0.188172    1.000000
C1---J0---R0       0.176344    1.000000
D1---J0---R0       0.176344    1.000000
F3---J0---R0       0.174194    1.000000
E1---J0---R0       0.172043    1.000000
E2---J0---R0       0.168817    1.000000
F4---J0---R0       0.153763    1.000000
A3---J0---R0       0.151613    1.000000
E3---J0---R0       0.147312    1.000000
A1---R0---J0       0.146237    1.000000
A1---J0---R0       0.146237    1.000

,support,confidence
J0---R0,0.548387,1.000000
H4---J0---R0,0.266667,1.000000
C2---J0---R0,0.240860,1.000000
D2---R0---J0,0.233333,1.000000
D2---J0---R0,0.233333,1.000000
A2---J0---R0,0.224731,1.000000
B2---R0---J0,0.217204,1.000000
B2---J0---R0,0.217204,1.000000
B1---J0---R0,0.193548,1.000000
R0---S4---J0,0.188172,1.000000
